# Notebook 02: Prompt Structure Patterns

**Objectives:**
- Compare unstructured vs structured prompts
- Use skeleton.v1 and zero_shot.v1 from central catalog
- Measure token efficiency and response quality

In [1]:
import sys
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model

## Unstructured vs Skeleton Pattern

Compare a vague prompt with a structured skeleton prompt.

In [3]:
model = pick_model('google', 'general')
client = LLMClient('google', model)

# Load context
with open('../data/tiny_corpus/02_product_specs.md') as f:
    context = f.read()[:1000]

# Approach 1: Unstructured prompt (baseline)
unstructured_prompt = f"Summarize this product:\n\n{context}"

response_unstructured = client.chat(
    [{'role': 'user', 'content': unstructured_prompt}], 
    temperature=0.2
)

print('=== UNSTRUCTURED PROMPT ===')
print(f'Tokens: {response_unstructured["usage"]["total_est"]}')
print(f'Result: {response_unstructured["text"]}\n')
log_llm_call('openai', model, 'unstructured', response_unstructured['latency_ms'], response_unstructured['usage'])

=== UNSTRUCTURED PROMPT ===
Tokens: 226
Result: CloudSync Pro 2.5.0, releasing in Q1 2025, is an enterprise-grade file synchronization and collaboration platform for distributed teams. Key features include real-time file syncing with low latency and adaptive bandwidth management, robust version control with history retention, comparison, rollback, and branching, and strong security and compliance measures. It also offers offline mode and batch processing capabilities.




In [4]:
# Approach 2: Skeleton pattern (structured)
prompt_text, spec = render(
    'skeleton.v1',
    role='Technical summarizer',
    task='Summarize CloudSync Pro in 3 sentences',
    context=context,
    constraints='Max 3 sentences, focus on value proposition',
    format='Plain text',
    checks='Exactly 3 sentences'
)

response_skeleton = client.chat(
    [{'role': 'user', 'content': prompt_text}], 
    temperature=0.2
)

print('=== SKELETON PATTERN ===')
print(f'Tokens: {response_skeleton["usage"]["total_est"]}')
print(f'Result: {response_skeleton["text"]}\n')
log_llm_call('openai', model, 'skeleton', response_skeleton['latency_ms'], response_skeleton['usage'])


=== SKELETON PATTERN ===
Tokens: 267
Result: CloudSync Pro is an enterprise-grade file synchronization and collaboration platform enabling secure, real-time file sharing for distributed teams. It features advanced version control with conflict resolution, ensuring data integrity and efficient collaboration. With robust security and compliance features, CloudSync Pro provides a reliable solution for businesses needing to manage and protect their data.




## Zero-Shot Pattern

Demonstrate the zero_shot.v1 pattern for classification tasks.


In [5]:
# Zero-shot classification example
customer_feedback = """I've been using CloudSync Pro for 3 months and I'm extremely disappointed. 
The sync is slow, files get corrupted, and customer support never responds."""

# Unstructured approach
unstructured_classify = f"Is this feedback positive or negative?\n\n{customer_feedback}"

response_unstructured_cls = client.chat(
    [{'role': 'user', 'content': unstructured_classify}], 
    temperature=0.0
)

print('=== UNSTRUCTURED CLASSIFICATION ===')
print(f'Result: {response_unstructured_cls["text"]}\n')

# Zero-shot structured approach
prompt_text, spec = render(
    'zero_shot.v1',
    role='customer feedback analyzer',
    instruction='Classify the sentiment of the customer feedback',
    constraints='Return only one word: positive, negative, or neutral',
    format='Single word'
)

full_prompt = f"{prompt_text}\n\nFeedback: {customer_feedback}"

response_zero_shot = client.chat(
    [{'role': 'user', 'content': full_prompt}], 
    temperature=0.0
)

print('=== ZERO-SHOT PATTERN ===')
print(f'Result: {response_zero_shot["text"]}\n')
log_llm_call('openai', model, 'zero_shot_classify', response_zero_shot['latency_ms'], response_zero_shot['usage'])


=== UNSTRUCTURED CLASSIFICATION ===
Result: This feedback is overwhelmingly **negative**.

Here's why:

*   **"Extremely disappointed"**: This is a strong negative sentiment.
*   **"Sync is slow"**: This is a performance issue.
*   **"Files get corrupted"**: This is a serious data integrity issue.
*   **"Customer support never responds"**: This is a customer service failure.

All of these points indicate a very poor user experience.


=== ZERO-SHOT PATTERN ===
Result: negative




## Comparison & Analysis

Compare token efficiency and response quality.


In [6]:
import pandas as pd

# Create comparison table
comparison = pd.DataFrame({
    'Approach': ['Unstructured', 'Skeleton Pattern', 'Zero-Shot Pattern'],
    'Use Case': ['Summarization', 'Summarization', 'Classification'],
    'Prompt Tokens (est)': [
        response_unstructured['usage']['input_tokens_est'],
        response_skeleton['usage']['input_tokens_est'],
        response_zero_shot['usage']['input_tokens_est']
    ],
    'Total Tokens (est)': [
        response_unstructured['usage']['total_est'],
        response_skeleton['usage']['total_est'],
        response_zero_shot['usage']['total_est']
    ],
    'Pros': [
        'Simple, fast to write',
        'Clear structure, better control',
        'Precise outputs, easy to parse'
    ],
    'Cons': [
        'Unpredictable outputs',
        'More verbose prompts',
        'Needs clear constraints'
    ]
})

print(comparison.to_string(index=False))

print('\n=== KEY INSIGHTS ===')
print(f'1. Skeleton pattern uses ~{response_skeleton["usage"]["input_tokens_est"] - response_unstructured["usage"]["input_tokens_est"]} more prompt tokens but provides better control')
print(f'2. Zero-shot pattern produces consistent, parseable outputs')
print(f'3. Structured patterns reduce need for retry/validation (lower total cost)')
print(f'4. Trade-off: Prompt complexity vs Output reliability')


         Approach       Use Case  Prompt Tokens (est)  Total Tokens (est)                            Pros                    Cons
     Unstructured  Summarization                  223                 226           Simple, fast to write   Unpredictable outputs
 Skeleton Pattern  Summarization                  264                 267 Clear structure, better control    More verbose prompts
Zero-Shot Pattern Classification                   79                  82  Precise outputs, easy to parse Needs clear constraints

=== KEY INSIGHTS ===
1. Skeleton pattern uses ~41 more prompt tokens but provides better control
2. Zero-shot pattern produces consistent, parseable outputs
3. Structured patterns reduce need for retry/validation (lower total cost)
4. Trade-off: Prompt complexity vs Output reliability


## Key Takeaways

1. **Structured prompts > Unstructured**: Higher upfront prompt tokens, but more reliable outputs
2. **Skeleton pattern**: Best for complex tasks requiring clear role, task, context, constraints
3. **Zero-shot pattern**: Perfect for classification tasks with predictable outputs
4. **Token efficiency**: Measured by total cost including retries, not just prompt tokens
5. **Central catalog**: Using `utils/prompts.py` ensures consistency across your codebase

**When to use each:**
- **Unstructured**: Quick experiments, exploratory queries
- **Skeleton**: Complex generation tasks, content creation, summarization
- **Zero-shot**: Classification, labeling, structured extraction

**Next:** `03_zero_few_cot_tot.ipynb`
